In [5]:
# Load Libraries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs

In [2]:
data = xr.open_mfdataset(r'H:\Downloads\N_MODISAqua Aerosol Cloud Water Vapor Ozone Daily L3 Global 1Deg CMG\adaptor.mars.internal-1708948417.9577672-21739-2-d2677091-5610-461b-8f4c-574028542a13.nc')
data

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, level: 20, time: 24)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 50 100 150 200 250 300 ... 750 800 850 900 950 1000
  * time       (time) datetime64[ns] 2023-06-01 ... 2023-06-06T18:00:00
Data variables:
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 20, 721, 1440), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 20, 721, 1440), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 20, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-02-26 11:53:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [3]:
# Slice for our time of interest
data1 = data.sel(level=slice(300, 1000))
del data
data1

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, level: 15, time: 24)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 300 350 400 450 500 550 ... 800 850 900 950 1000
  * time       (time) datetime64[ns] 2023-06-01 ... 2023-06-06T18:00:00
Data variables:
    q          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 15, 721, 1440), meta=np.ndarray>
    u          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 15, 721, 1440), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 dask.array<chunksize=(24, 15, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-02-26 11:53:56 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [6]:
# Calculating Integrated_waterVapour_Transport for different levels
levels = [14,13,12,11,10,9,8,7,6,5,4,3,2,1,0]
ivt = []
du = []
dv = []
for i in np.arange(0,len(data1.time)):
    for j in np.arange(0,len(levels)-1):
        u = (data1.u[i,j,:,:].to_numpy()+data1.u[i,j+1,:,:].to_numpy())/2
        du.append(u)
        qu= (np.mean(np.array(du))*data1.q[i,j,:,:].to_numpy())/9.81
        
        v = (data1.v[i,j,:,:].to_numpy()+data1.v[i,j+1,:,:].to_numpy())/2
        dv.append(v)
        qv= (np.mean(np.array(dv))*data1.q[i,j,:,:].to_numpy())/9.81
        ivt1 = np.square((qu)**2+(qv)**2)
    ivt.append(ivt1)

In [ ]:
# Calculate 85 Percentile and plot 
dates = ['2023-06-01T00','2023-06-01T06',
       '2023-06-01T12', '2023-06-01T18',
       '2023-06-02T00', '2023-06-02T06',
       '2023-06-02T12', '2023-06-02T18',
       '2023-06-03T00', '2023-06-03T06',
       '2023-06-03T12', '2023-06-03T18',
       '2023-06-04T00', '2023-06-04T06',
       '2023-06-04T12', '2023-06-04T18',
       '2023-06-05T00', '2023-06-05T06',
       '2023-06-05T12', '2023-06-05T18',
       '2023-06-06T00', '2023-06-06T06',
       '2023-06-06T12', '2023-06-06T18']

ind = np.arange(0,24)

for i in ind:
    fname= r'F:\Indian Shape Files\Indian Administrative bourders\IND_adm\IND_adm1.shp'
    adm1_shapes = list(shpreader.Reader(fname).geometries())
    fname2 = r'F:\Indian Shape Files\Only Indian Boundaries\India_Boundary\India_Boundary.shp'
    adm1_shapes2 = list(shpreader.Reader(fname2).geometries())

    plt.clf()
    fig=plt.figure(figsize=(15,12))#,constrained_layout=True) 
    ax = fig.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.8,wspace=0.02, hspace=0.02)


    def main():
        ax1 = plt.axes(projection=ccrs.PlateCarree())  
        ax1.set_extent([30,140,-30,55], ccrs.PlateCarree())     
        ax1.set_title(f'Integrated water vapor transport on {dates[i]}')
        
        # Calculating 85 percentile 
        percentile = np.percentile(ivt2[i,:,:], 90)
        ivt2[i,:,:][ivt2[i,:,:] < percentile] = np.nan 
        
        filled_c = ax1.contourf(data1.longitude, data1.latitude, ivt2[i,:,:],
                                60, cmap='jet',vmin=0, vmax=1.10e-08,transform=ccrs.PlateCarree())
        ax1.quiver(data1.longitude,data1.latitude,np.array(data1.u[i,11,:,:]),np.array(data1.v[i,11,:,:]),regrid_shape=25)#
#                 scale=100,scale_units='xy')
        ax1.coastlines('50m')
#     ax1.add_geometries(adm1_shapes, ccrs.PlateCarree(),
#                   edgecolor='black', alpha=0.9,lw = 0.5, facecolor='none') #
        ax1.add_geometries(adm1_shapes2, ccrs.PlateCarree(),
                  edgecolor='black', alpha=0.9,lw = 0.5, facecolor='none') #
        ax1.text(0.27, 0.1, dates[i], horizontalalignment='center', verticalalignment='center',
                 fontweight="bold", size=10,family= 'Arial',transform=ax1.transAxes)
        a = plt.colorbar(filled_c,ax=ax1,orientation ='vertical', pad = 0.05,shrink=0.75)
    
    if __name__ == '__main__':
        main()
    plt.savefig(r'H:\Downloads\N_MODISAqua Aerosol Cloud Water Vapor Ozone Daily L3 Global 1Deg CMG\\'+dates[i]+'_90_per_.jpg',
            bbox_inches='tight',dpi=180)
    
